In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

ANN Prototype

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
    layers.Input(shape = (128,)),
    layers.Normalization(mean = 127, variance = 77.4),
    layers.Dense(100, activation='tanh'), #86
    layers.Dense(75, activation='tanh'), #64
    layers.Dense(50, activation='tanh'), #42
    layers.Dense(25, activation='tanh'), #21
    layers.Dense(1, activation='relu')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dense_1 (Dense)              (None, 75)                7575      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 26        
Total params: 25,576
Trainable params: 25,576
Non-trainable params: 0
____________________________________________________

Test training

In [7]:
save_to_disk = True # Set to FALSE if we don't want this to overwrite the current model.

TrainFeatures = pd.read_csv('../data/trainFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets = pd.read_csv('../data/trainTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets.iloc[:,0] = TrainTargets.iloc[:,0].astype(float)

EvalFeatures = pd.read_csv('../data/evalFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets = pd.read_csv('../data/evalTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets.iloc[:,0] = EvalTargets.iloc[:,0].astype(float)

# Since our dataset is imbalanced, we need a weighted binary cross entropy loss function.
def weighted_bce(y_true, y_pred):
    weights = (y_true * 3.) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = tf.math.reduce_mean(bce * weights)
    return weighted_bce

# Here, we choose the loss function to minimize, the optimizer algorithm, and additional loss metrics to display on screen.
model.compile(loss = weighted_bce, optimizer = 'adam', metrics = ['accuracy'])

# Time to train! Epochs = # times the training algorithm goes through the whole set.
# Batch size = # elements the algorithm "sees" at a time between adjusting model parameters.
model.fit(TrainFeatures,TrainTargets,epochs = 10, batch_size = 8)

if save_to_disk:
    # Saving model to JSON and weights to H5.
    model_json = model.to_json()
    with open("../models/model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("../models/model.h5")
    print("Saved model to disk")

# Measure accuracy on the evaluation set.
loss  = model.evaluate(EvalFeatures, EvalTargets)
print('Loss on Validation Set: ', loss)
print('Confusion matrix:')
predictions = np.round(model.predict(EvalFeatures))
true_labels = EvalTargets.iloc[:,0]

m = tf.keras.metrics.Recall()
m.update_state(true_labels, predictions)
print('Recall: ' + str(m.result().numpy()))

conf_matrix = tf.math.confusion_matrix(true_labels, predictions)
conf_matrix

Epoch 1/10
7993/7993 [==============================] - 7s 846us/step - loss: 2.4299 - accuracy: 0.7663
Epoch 2/10
7993/7993 [==============================] - 7s 833us/step - loss: 1.0978 - accuracy: 0.7851
Epoch 3/10
7993/7993 [==============================] - 7s 862us/step - loss: 0.7784 - accuracy: 0.8092
Epoch 4/10
7993/7993 [==============================] - 7s 838us/step - loss: 0.6996 - accuracy: 0.8190
Epoch 5/10
7993/7993 [==============================] - 6s 801us/step - loss: 0.6961 - accuracy: 0.8192
Epoch 6/10
7993/7993 [==============================] - 6s 811us/step - loss: 0.6505 - accuracy: 0.8256
Epoch 7/10
7993/7993 [==============================] - 7s 827us/step - loss: 0.6502 - accuracy: 0.8329
Epoch 8/10
7993/7993 [==============================] - 6s 811us/step - loss: 0.6460 - accuracy: 0.8333
Epoch 9/10
7993/7993 [==============================] - 7s 829us/step - loss: 0.6249 - accuracy: 0.8421
Epoch 10/10
7993/7993 [==============================] - 7s 827u

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[9408,  817],
       [1699,  865]])>